# **ROB 535 Perception Task 1**

**1) Importing Libraries**

In [1]:
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Dense, Dropout

from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
% cd /content/gdrive/My Drive/ROB535-Perception-Project-1

/content/gdrive/My Drive/ROB535-Perception-Project-1


In [4]:
train = pd.read_csv("task-1-training-dataset-old/train_labels.csv")
validation = pd.read_csv("task-1-training-dataset-old/test_labels.csv")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6815 entries, 0 to 6814
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  6815 non-null   object
 1   width     6815 non-null   int64 
 2   height    6815 non-null   int64 
 3   class     6815 non-null   object
 4   xmin      6815 non-null   int64 
 5   ymin      6815 non-null   int64 
 6   xmax      6815 non-null   int64 
 7   ymax      6815 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 426.1+ KB


In [6]:
train_folder = "task-1-training-dataset-old/cropped_images/train"
validation_folder = "task-1-training-dataset-old/cropped_images/test"
test_folder = "test"

In [7]:
# Data generator to augment train data
train_gen = ImageDataGenerator(
rotation_range=45,
rescale=1./255,
horizontal_flip=True
)
validation_gen = ImageDataGenerator(rescale = 1.255)

test_gen = ImageDataGenerator(rescale = 1.255)

In [8]:
train_data = train_gen.flow_from_dataframe(dataframe = train, 
directory = train_folder, x_col = "filename", 
y_col = "class", seed = 42,
shuffle = True, 
class_mode="categorical",
target_size = (300, 300))

validation_data = validation_gen.flow_from_dataframe(dataframe = validation, 
directory = validation_folder, x_col = "filename", 
y_col = "class",
shuffle = False, 
class_mode="categorical",
target_size = (300, 300))

test_data = test_gen.flow_from_directory(test_folder, target_size=(300, 300), class_mode=None, shuffle=False)

Found 6815 validated image filenames belonging to 3 classes.
Found 758 validated image filenames belonging to 3 classes.
Found 2631 images belonging to 23 classes.


In [9]:
# Xception Model

# base_model = keras.applications.Xception(
#     weights='imagenet',  
#     # input_shape=(150, 150, 3),
#     input_shape=(300, 300, 3),
#     include_top=False)  


In [10]:
base_model = keras.applications.ResNet50V2(
    weights=None,  
    # input_shape=(150, 150, 3),
    input_shape=(300, 300, 3),
    include_top=False)  


In [11]:
base_model.trainable = True

In [12]:
inputs = keras.Input(shape=(300, 300, 3))

In [13]:
# x = tf.keras.applications.xception.preprocess_input(inputs)

In [14]:
x = tf.keras.applications.resnet50.preprocess_input(inputs)

In [15]:
x = base_model(x, training=True)
# x = keras.layers.GlobalAveragePooling2D()(x)
# # x = keras.layers.Dense(1024, activation=("relu"))(x)
# x = keras.layers.Dense(512, activation=("relu"))(x)
# x = keras.layers.Dense(256, activation=("relu"))(x)
# x = keras.layers.Dropout(0.5)(x)
# x = keras.layers.Dense(128, activation=("relu"))(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(3, activation=('softmax'))(x)
model = keras.Model(inputs, outputs)

# x = keras.layers.GlobalAveragePooling2D()(x)
# x = keras.layers.Dropout(0.2)(x)  
# outputs = keras.layers.Dense(3, activation=('softmax'))(x)
# model = keras.Model(inputs, outputs)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 300, 300, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 300, 300, 3)      0         
                                                                 
 resnet50v2 (Functional)     (None, 10, 10, 2048)      23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0     

In [17]:
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical

In [18]:
batch_size= 100
epochs=20
learn_rate=.001
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [19]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [20]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath='training-checkpoints/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.LearningRateScheduler(scheduler)
]

In [ ]:
model.fit(train_data, validation_data=validation_data, epochs=epochs, callbacks=my_callbacks)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
  8/213 [>.............................] - ETA: 1:55:36 - loss: 1.1637 - accuracy: 0.4336

In [ ]:
predictions = model.predict(test_data)

In [ ]:
model.save("model-resnet-4")

In [ ]:
import numpy as np

In [ ]:
np.argmax(predictions,axis=1)

In [ ]:
outputs = np.argmax(predictions,axis=1)

In [ ]:
some_model = keras.models.load_model("model-resnet-3")

In [ ]:
arr = np.unique(np.argmax(predictions,axis=1))

In [ ]:
arr

In [ ]:
df = pd.DataFrame(outputs)
df.to_csv('/content/gdrive/My Drive/ROB535-Perception-Project-1/prediction_outputs4.csv')